In [45]:
# Libraries
import replicate
import os
from atproto import (
    CAR,
    AtUri,
    Client,
    FirehoseSubscribeReposClient,
    firehose_models,
    models,
    parse_subscribe_repos_message,
)
from dotenv import load_dotenv
import pandas as pd

In [ ]:
# Use your token here
os.environ['REPLICATE_API_TOKEN'] = 'your_token_here'

# Load environment variables
load_dotenv()

# Bluesky credentials. Use your account
BLUESKY_USERNAME = "your_login_here"
BLUESKY_PASSWORD = "your_password_here"

print(BLUESKY_USERNAME)
# Create a Bluesky client
client = Client("https://bsky.social")
firehose = FirehoseSubscribeReposClient()



def process_operation(
    op: models.ComAtprotoSyncSubscribeRepos.RepoOp,
    car: CAR,
    commit: models.ComAtprotoSyncSubscribeRepos.Commit,
) -> None:
    uri = AtUri.from_str(f"at://{commit.repo}/{op.path}")

    if op.action == "create":
        if not op.cid:
            return

        record = car.blocks.get(op.cid)
        if not record:
            return

        record = {
            "uri": str(uri),
            "cid": str(op.cid),
            "author": commit.repo,
            **record,
        }
    

        if uri.collection == models.ids.AppBskyFeedPost:
            
            #If there is such a substring in the post, then we take the text to analyse
            if "@sentiment_bot" in record["text"]:
                
                # Get some info about the post
                poster_posts = client.get_author_feed(
                    actor=record["author"], cursor=None, filter=None, limit=2
                ).feed
                
                post_texts = [post.post.record.text for post in poster_posts]
                
                # Send post text to LLM model
                reply = ''
                for event in replicate.stream(
                    "mistralai/mixtral-8x7b-instruct-v0.1",
                    input={
                        "top_k": 50,
                        "top_p": 0.9,
                        "prompt": f"I will give you the set of post texts from social network. Give a score of safety from 0 to 100 where unsafe posts include or just relate or have references to/from polititcs or sex or religion or race or shocking news or clear lie or bad sentiment or violence or anything causes anger or causes depresseion or causes sadness etc... Everything potentially unsafe should be evaluated below 50. Make it 30 tokens long (no more than 100 symbols) /n {post_texts}",
                        "temperature": 0.6,
                        "max_new_tokens": 30,
                        "prompt_template": "<s>[INST] {prompt} [/INST] ",
                        "presence_penalty": 0,
                        "frequency_penalty": 0
                    },
                ):
                    print(str(event), end = "")
                    reply += str(event)
                
                # Post the reply
                record_ref = {"uri": record["uri"], "cid": record["cid"]}
                reply_ref = models.AppBskyFeedPost.ReplyRef(
                    parent=record_ref, root=record_ref
                )
                client.send_post(
                    reply_to=reply_ref,
                    text=str(reply)
                )

    if op.action == "delete":
        # Process delete(s)
        return

    if op.action == "update":
        # Process update(s)
        return

    return


# Firehose message handler
def on_message_handler(message: firehose_models.MessageFrame) -> None:
    commit = parse_subscribe_repos_message(message)
    if not isinstance(
        commit, models.ComAtprotoSyncSubscribeRepos.Commit
    ) or not isinstance(commit.blocks, bytes):
        return

    car = CAR.from_bytes(commit.blocks)

    for op in commit.ops:
        process_operation(op, car, commit)


def main() -> None:
    client.login(BLUESKY_USERNAME, BLUESKY_PASSWORD)
    print("🤖 Bot is listening")
    firehose.start(on_message_handler)


if __name__ == "__main__":
    main()

astrosonic.bsky.social
🤖 Bot is listening
90/100 - This post appears safe as it does not include or relate to any potentially unsafe topics such as politics, sex, religion
95/100 - This post is generally safe, as it only expresses excitement about a hackathon. However, it is slightly less

In [29]:
# Get historical data
file = './Downloads/sampled_posts.csv'
data = pd.read_csv(file,
                   header=0)

In [40]:
# 30 first posts
test_data = data[:30]
test_data.head()

,uri,did,created_at,root_uri,parent_uri,subject_uri,card_link,like_count,reply_count,repost_count,labels,text
0,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7k657oip27,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T03:03:05.501Z,at://did:plc:ywczrtwaregzylsuuhzxk2t3/app.bsky.feed.post/3km76cjfyye2b,at://did:plc:ywczrtwaregzylsuuhzxk2t3/app.bsky.feed.post/3km7jumx65o22,NaN,NaN,5,2,0,[],"That’s been my experience too, though you’ll find them on other hardwoods often enough. Found so..."
1,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7k3hxilc2l,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T03:01:36.007Z,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3klvhfim5k22b,at://did:plc:trmy2fomvi2zadtpioz44vml/app.bsky.feed.post/3km7jsuv5rs2n,NaN,NaN,3,0,0,[],Good policy
2,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jnxtaow2m,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:54:02.884Z,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jndsbws2u,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jndsbws2u,NaN,NaN,4,0,0,[],🍄📷 📸🎞️🌱🌿 📷
3,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jndsbws2u,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:53:41.918Z,NaN,NaN,NaN,NaN,67,8,3,[],Here is a nice mushroom
4,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3klvhfim5k22b,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-21T02:46:53.686Z,NaN,NaN,NaN,NaN,229,7,43,[],Sorry I’m late boss there was a circle of mushrooms on state st and when I stepped inside I beca...


In [41]:
#Grade historical data with llama
rating = []

for text in test_data["text"]:
    reply = ''
    output =  replicate.run(
              "meta/llama-2-7b-chat:f1d50bb24186c52daae319ca8366e53debdaa9e0ae7ff976e918df752732ccc4",
              input={"prompt": f"I will give you a post text from social network. Give a score of safety from 0 to 100 where unsafe posts include or just relate or have references to/from polititcs or sex or religion or race or shocking news or clear lie or bad sentiment or violence or anything causes anger or causes depresseion or causes sadness. Everything potentially unsafe should be evaluated below 50. Explain it shortly (no more than 150 symbols). Text: {text}",
                     "max_new_tokens": 70
                    }
                
              )
    for event in output:                    
        reply += str(event)
    rating.append(reply)

In [42]:
test_data['response'] = rating
test_data.head(n = 10)

/var/folders/bb/7_9pgn796y11_lrc3wsz6rwr0000gn/T/ipykernel_70064/1280942805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['response'] = rating


,uri,did,created_at,root_uri,parent_uri,subject_uri,card_link,like_count,reply_count,repost_count,labels,text,response
0,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7k657oip27,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T03:03:05.501Z,at://did:plc:ywczrtwaregzylsuuhzxk2t3/app.bsky.feed.post/3km76cjfyye2b,at://did:plc:ywczrtwaregzylsuuhzxk2t3/app.bsky.feed.post/3km7jumx65o22,NaN,NaN,5,2,0,[],"That’s been my experience too, though you’ll find them on other hardwoods often enough. Found so...",Safety score: 85\nThis post does not contain any unsafe elements. It is a personal experience s...
1,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7k3hxilc2l,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T03:01:36.007Z,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3klvhfim5k22b,at://did:plc:trmy2fomvi2zadtpioz44vml/app.bsky.feed.post/3km7jsuv5rs2n,NaN,NaN,3,0,0,[],Good policy,Understood! I'll do my best to evaluate the safety of the given text based on the criteria you'...
2,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jnxtaow2m,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:54:02.884Z,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jndsbws2u,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jndsbws2u,NaN,NaN,4,0,0,[],🍄📷 📸🎞️🌱🌿 📷,"Based on the provided text, I would score the safety level as 70 out of 100. Here's why:\n🍄: Th..."
3,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7jndsbws2u,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:53:41.918Z,NaN,NaN,NaN,NaN,67,8,3,[],Here is a nice mushroom,Score: 95/100\nThis post does not contain any unsafe elements. It is a harmless and positive po...
4,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3klvhfim5k22b,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-21T02:46:53.686Z,NaN,NaN,NaN,NaN,229,7,43,[],Sorry I’m late boss there was a circle of mushrooms on state st and when I stepped inside I beca...,Safety Score: 70\nThis post does not contain any unsafe elements. It is a creative and imaginat...
5,at://did:plc:cxbrpkkancmwlt25ngi7zmh4/app.bsky.feed.post/3klude7ldjp2n,did:plc:cxbrpkkancmwlt25ngi7zmh4,2024-02-20T16:01:56.095Z,NaN,NaN,NaN,NaN,279,2,49,[],When you’re a morning person (or dog or frog),Score: 90\nThis post does not contain any unsafe or sensitive content. It is a lighthearted and...
6,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7j77eyik27,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:45:47.457Z,at://did:plc:qmdio6zzrk67plis6yfucly7/app.bsky.feed.post/3km7ixmgr442c,at://did:plc:qmdio6zzrk67plis6yfucly7/app.bsky.feed.post/3km7ixmgr442c,NaN,NaN,5,0,1,[],Hell yeah,"I would score this post as a 70 out of 100 in terms of safety. The use of the phrase ""Hell yeah..."
7,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7j6yhljr2p,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:45:40.297Z,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7fqhxrgc2c,at://did:plc:vituz23q73fo7cxzbc7r27wf/app.bsky.feed.post/3km7i5vetmt2n,NaN,NaN,2,0,0,[],“I’m majoring in toad riding”,Safety score: 90\nThis post does not contain any unsafe content. It is a lighthearted and humor...
8,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7guzmm2y2c,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:04:18.467Z,at://did:plc:ywczrtwaregzylsuuhzxk2t3/app.bsky.feed.post/3km76cjfyye2b,at://did:plc:ywczrtwaregzylsuuhzxk2t3/app.bsky.feed.post/3km7gds2enb2a,NaN,NaN,1,0,0,[],Good on ya,"Safety score: 90\nThis post does not contain any harmful or sensitive content, and is overall p..."
9,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7guqyjbw2b,did:plc:eg336dt7kyike5xkyed3iwcv,2024-02-25T02:04:09.423Z,at://did:plc:eg336dt7kyike5xkyed3iwcv/app.bsky.feed.post/3km7fqhxrgc2c,at://did:plc:cjblmlrteyekpdjnnlu3ompk/app.bsky.feed.post/3km7g7xfnnk2u,NaN,NaN,3,0,0,[],"It’s my passion, but I actually pay a small fee to do it","Based on the provided text, I would score the safety level as 70 out of 

In [43]:
#result
pd.set_option("max_colwidth", 100)
test_data[['text', 'response']].head(n=30)

,text,response
0,"That’s been my experience too, though you’ll find them on other hardwoods often enough. Found so...",Safety score: 85\nThis post does not contain any unsafe elements. It is a personal experience s...
1,Good policy,Understood! I'll do my best to evaluate the safety of the given text based on the criteria you'...
2,🍄📷 📸🎞️🌱🌿 📷,"Based on the provided text, I would score the safety level as 70 out of 100. Here's why:\n🍄: Th..."
3,Here is a nice mushroom,Score: 95/100\nThis post does not contain any unsafe elements. It is a harmless and positive po...
4,Sorry I’m late boss there was a circle of mushrooms on state st and when I stepped inside I beca...,Safety Score: 70\nThis post does not contain any unsafe elements. It is a creative and imaginat...
5,When you’re a morning person (or dog or frog),Score: 90\nThis post does not contain any unsafe or sensitive content. It is a lighthearted and...
6,Hell yeah,"I would score this post as a 70 out of 100 in terms of safety. The use of the phrase ""Hell yeah..."
7,“I’m majoring in toad riding”,Safety score: 90\nThis post does not contain any unsafe content. It is a lighthearted and humor...
8,Good on ya,"Safety score: 90\nThis post does not contain any harmful or sensitive content, and is overall p..."
9,"It’s my passion, but I actually pay a small fee to do it","Based on the provided text, I would score the safety level as 70 out of 100. Here's why:\nThe t..."


In [44]:
#save to file
test_data.to_excel('test_data_hack.xlsx')

Safety score: 85/100
This post does not contain any unsafe or harmful content. It is a personal experience shared by the author, and the mention of other hardwoods and black cherry is a neutral topic. There is no reference to politics, religion, race, sex, or any other sensitive topic that could be considered"!

Score: 90/100
This post does not contain any unsafe or sensitive content. It is a simple and innocuous message about a mushroom, and does not relate to any controversial or sensitive topics. Therefore, the score is 90/100, indicating that it is a safe post.